In [4]:
print("Hello world!")

Hello world!


In [5]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM, Dropout, Dense, InputLayer
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import glob
import os

In [8]:
# file_path = '/path/to/Classical' 각자가 저장한 폴더로 연결해주세요
file_path = './datasets/datasets/Classical/*.npz'
file_list = glob.glob(file_path)
print(file_list)

['./datasets/datasets/Classical/57_DutchPowerDemand.npz', './datasets/datasets/Classical/55_Boschline.npz', './datasets/datasets/Classical/39_HSEFilters2.npz', './datasets/datasets/Classical/45_SWAT.npz', './datasets/datasets/Classical/62_ECoating.npz', './datasets/datasets/Classical/65_MachineryFault.npz', './datasets/datasets/Classical/43_Motorcondition2.npz', './datasets/datasets/Classical/54_CNCMachining.npz', './datasets/datasets/Classical/42_Motorcondition1.npz', './datasets/datasets/Classical/50_PLAID.npz', './datasets/datasets/Classical/48_Ladlefurnace.npz', './datasets/datasets/Classical/60_Concrete.npz', './datasets/datasets/Classical/49_Wafer2.npz', './datasets/datasets/Classical/35_IMS.npz', './datasets/datasets/Classical/66_Cuttingblade.npz', './datasets/datasets/Classical/38_HSEFilters1.npz', './datasets/datasets/Classical/51_PowerCons.npz', './datasets/datasets/Classical/63_Prensas.npz', './datasets/datasets/Classical/40_Yahoo1.npz', './datasets/datasets/Classical/36_PHM

In [14]:
data = np.load('datasets/datasets/Classical/65_MachineryFault.npz', allow_pickle=True)
# data = np.load('datasets/datasets/Classical/54_CNCMachining.npz', allow_pickle=True)
X, y = data['X'], data['y']

In [16]:
# X shape : 99399, 3
# (Batch size, Sequence(timestamp), Feature)

X[0:5]
X.shape
X

array([[ 4.4359  , -1.1073  ,  0.22269 , ..., -0.037787,  0.11994 ,
        -0.14026 ],
       [ 4.4628  ,  0.47153 ,  0.049814, ..., -0.037627,  0.14496 ,
         0.15596 ],
       [ 4.4511  , -1.8167  ,  0.054311, ..., -0.037447,  0.10992 ,
        -0.045535],
       ...,
       [-0.53822 ,  1.2746  ,  0.43657 , ..., -0.051854, -2.1676  ,
        -0.13792 ],
       [-0.52845 , -0.88213 , -0.25662 , ..., -0.052278, -2.3157  ,
         0.20938 ],
       [-0.53948 ,  0.76996 ,  0.38191 , ..., -0.053375, -2.2277  ,
        -0.13198 ]])

In [11]:
y

array([0., 0., 0., ..., 1., 1., 1.])

In [ ]:
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout
from tensorflow.keras.models import Model

class ConvLSTMModel(Model):
    def __init__(self, in_channel):
        super(ConvLSTMModel, self).__init__()
        self.conv1d_1 = Conv1D(filter=16, kernel_size=3, strides=1, padding='same', input_shape=in_channel)

        self.conv1d_2 = Conv1D(filter=32, kernel_size=3, strides=1, padding='same', input_shape=16)

        self.lstm = LSTM()
    


In [34]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, LSTM, Dropout, Dense
from tensorflow.keras import Model
tf.config.run_functions_eagerly(True)

class Conv1d_LSTM(Model):
    def __init__(self, in_channel=3, out_channel=1):
        super(Conv1d_LSTM, self).__init__()
        self.conv1d_1 = Conv1D(filters=16,
                               kernel_size=3,
                               strides=1,
                               padding='same',
                               activation=None)  # Activation will be added separately
        self.conv1d_2 = Conv1D(filters=32,
                               kernel_size=3,
                               strides=1,
                               padding='same',
                               activation=None)  # Activation will be added separately
        
        self.lstm = LSTM(units=50,
                         return_sequences=True,
                         return_state=False,
                         dropout=0.0,
                         recurrent_dropout=0.0)
        
        self.dropout = Dropout(0.5)
        
        self.dense1 = Dense(32, activation='relu')
        self.dense2 = Dense(out_channel)
    @tf.function
    def call(self, inputs, training=False):
        x = inputs
        x = tf.transpose(x, perm=[0, 2, 1])  # Shape: (B, S, F) => (B, F, S)
        x = self.conv1d_1(x)
        x = tf.nn.relu(x)  # Applying ReLU activation after Conv1D
        x = self.conv1d_2(x)
        x = tf.nn.relu(x)  # Applying ReLU activation after Conv1D
        x = tf.transpose(x, perm=[0, 2, 1])  # Shape: (B, F, S) => (B, S, F)
        
        x, _ = self.lstm(x)  # x shape: (B, S, H) => (B, 10, 50)
        
        x = self.dropout(x, training=training)
        
        x = self.dense1(x[:, -1, :])  # Take the last sequence element
        x = self.dense2(x)
        
        return x


In [35]:
model = Conv1d_LSTM(in_channel=3, out_channel=1)

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',  # 회귀 문제의 경우
              metrics=['accuracy'])  # 분류 문제의 경우 accuracy로 변경 가능

In [36]:
# 훈련 데이터 예제
import numpy as np

# 예제 데이터 생성
X_train = np.random.rand(100, 10, 3)  # 100개의 샘플, 각 샘플은 (10, 3) 모양
y_train = np.random.rand(100, 1)  # 회귀 문제의 경우

# 모델 훈련
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2,  # 20%의 데이터는 검증용으로 사용
                    verbose=1)


Epoch 1/10


/Users/seyeong/anaconda3/envs/anomaly_detection/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


ValueError: Exception encountered when calling Conv1d_LSTM.call().

[1mtoo many values to unpack (expected 2)[0m

Arguments received by Conv1d_LSTM.call():
  • inputs=tf.Tensor(shape=(32, 10, 3), dtype=float32)
  • training=True

In [38]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, LSTM, Dropout, Dense
from tensorflow.keras import Model

class Conv1d_LSTM(Model):
    def __init__(self, in_channel=3, out_channel=1):
        super(Conv1d_LSTM, self).__init__()
        self.conv1d_1 = Conv1D(filters=16,
                               kernel_size=3,
                               strides=1,
                               padding='same',
                               activation=None)  # Activation will be added separately
        self.conv1d_2 = Conv1D(filters=32,
                               kernel_size=3,
                               strides=1,
                               padding='same',
                               activation=None)  # Activation will be added separately
        LSTM(units=50, input_shape=(X_train.shape[1], 1), return_sequences=True)
        self.lstm = LSTM(units=50, input_shape, return_sequences=True)
        self.lstm = LSTM(units=50,
                         return_sequences=True,
                         return_state=False,
                         dropout=0.0,
                         recurrent_dropout=0.0)
        
        self.dropout = Dropout(0.5)
        
        self.dense1 = Dense(32, activation='relu')
        self.dense2 = Dense(out_channel)

    def call(self, inputs, training=False):
        x = inputs
        x = tf.transpose(x, perm=[0, 2, 1])  # Shape: (B, S, F) => (B, F, S)
        x = self.conv1d_1(x)
        x = tf.nn.relu(x)  # Applying ReLU activation after Conv1D
        x = self.conv1d_2(x)
        x = tf.nn.relu(x)  # Applying ReLU activation after Conv1D
        x = tf.transpose(x, perm=[0, 2, 1])  # Shape: (B, F, S) => (B, S, F)
        
        x, _ = self.lstm(x)  # x shape: (B, S, H)
        
        x = self.dropout(x, training=training)
        
        x = self.dense1(x[:, -1, :])  # Take the last sequence element
        x = self.dense2(x)
        
        return x

# 모델 인스턴스 생성
model = Conv1d_LSTM(in_channel=3, out_channel=1)

# 모델 컴파일
model.compile(optimizer='adam',
              loss='mean_squared_error',  # 회귀 문제에 적합한 손실 함수
              metrics=['mae'])

# 예제 데이터 생성
import numpy as np
X_train = np.random.rand(100, 10, 3)  # (batch_size, sequence_length, features)
y_train = np.random.rand(100, 1)  # 회귀 문제의 경우

# 모델 훈련
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2,  # 20%의 데이터는 검증용으로 사용
                    verbose=1)


Epoch 1/10


ValueError: Exception encountered when calling Conv1d_LSTM.call().

[1mtoo many values to unpack (expected 2)[0m

Arguments received by Conv1d_LSTM.call():
  • inputs=tf.Tensor(shape=(32, 10, 3), dtype=float32)
  • training=True